# Chapter3-2 機械学習
## 3-2-1 scikit-learn

In [ ]:
import sklearn

## 3-2-2 データセットの準備

In [ ]:
from sklearn.datasets import load_breast_cancer

In [ ]:
brest_cancer = load_breast_cancer()

In [ ]:
type(brest_cancer)

In [ ]:
x = brest_cancer['data']
t = brest_cancer['target']

In [ ]:
type(x), x.shape

In [ ]:
type(t), t.shape

In [ ]:
# 先頭の 3 件を表示
x[:3]

In [ ]:
t

In [ ]:
len(t[t == 0]), len(t[t == 1])

In [ ]:
import numpy as np
# tが1の0の場合は1として、それ以外は0とする
t = np.where(t==0, 1, 0)

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

In [ ]:
plt.hist(t, bins=3)

## 3-2-2-1 訓練データとテストデータ

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
# テストが全体の　20%　となるようにランダムに分割
x_train, x_test, t_train, t_test = train_test_split(x, t, test_size=0.2, random_state=0)

In [ ]:
x_train.shape, x_test.shape

In [ ]:
x_train.shape, t_test.shape

## 3-2-3 モデルの訓練

In [ ]:
from sklearn.tree import DecisionTreeClassifier

In [ ]:
# モデルを clf としてインスタンス化
clf = DecisionTreeClassifier(random_state=0)

In [ ]:
# 訓練データに基づいて、パラメータの訓練
clf.fit(x_train, t_train)

In [ ]:
# テストデータに対する予測値の計算（検証時に使用）
y_test = clf.predict(x_test)

## 3-2-4 モデルの検証

### 3-2-4-1 正解率

In [ ]:
# テストデータに対する正解率の算出
clf.score(x_test, t_test)

In [ ]:
# 訓練データに対する正解率の算出
clf.score(x_train, t_train)

### 3-2-4-2 混合行列

In [ ]:
from sklearn.metrics import confusion_matrix

In [ ]:
matrix = confusion_matrix(t_test, y_test)
matrix

In [ ]:
import seaborn as sns

In [ ]:
# ヒートマップで表示
sns.heatmap(matrix, annot=True, cmap='Blues');
plt.xlabel('Prediction')
plt.ylabel('Target')

### 3-2-4-3 適合率

In [ ]:
from sklearn.metrics import precision_score

In [ ]:
precision_score(t_test, y_test)

### 3-2-4-4 再現率

In [ ]:
from sklearn.metrics import recall_score

In [ ]:
recall_score(t_test, y_test)

### 3-2-4-5 F 値

In [ ]:
from sklearn.metrics import f1_score

In [ ]:
f1_score(t_test, y_test)

## 3-2-5 再現率を高めるための工夫

In [ ]:
len(t[t == 0]), len(t[t == 1])

In [ ]:
weight = {
    0: len(t) / len(t[t==0]),
    1: len(t) / len(t[t==1])
}

In [ ]:
weight

In [ ]:
clf = DecisionTreeClassifier(random_state=0, class_weight=weight)

In [ ]:
# モデルの訓練
clf.fit(x_train, t_train)

In [ ]:
# 正解率
clf.score(x_test, t_test)

In [ ]:
# 予測値の計算
y_test = clf.predict(x_test)

In [ ]:
# 適合率（precision_score）と再現率（recall_score）を確認
precision_score(t_test, y_test), recall_score(t_test, y_test)

## 3-2-5-2 ハイパーパラメータのチューニング

In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
# ハイパーパラメータの候補を列挙
params = {
    "max_depth": list(range(2, 10)),
    "criterion": ["gini", "entropy"]
}

In [ ]:
# scoring は指定しない場合 Accuracy となります
# cv は交差検証の分割数
clf_grid = GridSearchCV(
    estimator = DecisionTreeClassifier(class_weight=weight, random_state=0),
    scoring = 'recall',
    param_grid = params,
    cv = 5,
)

In [ ]:
# すべてのハイパーパラメータの候補で訓練
clf_grid.fit(x_train, t_train)

In [ ]:
# 最も良かったハイパーパラメータの組み合わせ
clf_grid.best_params_

In [ ]:
# 最も良かったハイパーパラメータを持つ訓練済みモデルを受け継ぐ
clf = clf_grid.best_estimator_
clf

In [ ]:
# 正解率
clf.score(x_test, t_test)

In [ ]:
# 予測値の計算
y_test = clf.predict(x_test)

In [ ]:
# 適合率と再現率、F値
precision_score(t_test, y_test), recall_score(t_test, y_test), f1_score(t_test, y_test)

In [ ]:
# 混合行列
matrix = confusion_matrix(t_test, y_test)
matrix

In [ ]:
# 混合行列の可視化
sns.heatmap(matrix, annot=True, cmap='Blues');
plt.xlabel('Prediction')
plt.ylabel('Target')

## 3-2-6 訓練済みモデルの保存

In [ ]:
import joblib

In [ ]:
# brest_cancer.pkl という名前で保存
joblib.dump(clf, 'brest_cancer.pkl')

In [ ]:
clf_load = joblib.load('brest_cancer.pkl')

In [ ]:
# 読み込んだ訓練済みモデルを使用した推論
clf_load.predict(x_test)